In [10]:
import nltk
#nltk.download()
import os
import pandas as pd
# import numpy
# from os import listdir
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
#import matplotlib.pyplot as plt
import csv

In [11]:
######################################################## download data from the website
# import requests
# data_url = "https://www.kaggle.com/crowdflower/twitter-user-gender-classification/downloads/gender-classifier-DFE-791531.csv"
# kaggle_info = {'UserName': "avevanes", 'Password': "zqPwpz0t?"}
# r = requests.get(data_url)
# r = requests.post(r.url, data=kaggle_info)
# f = open('gender-classifier-DFE-791531.csv','w')
# for chunk in r.iter_content(chunk_size=512 * 1024):
#     if chunk:
#         f.write(chunk)
# f.close()



In [12]:
# clean
stop_words = stopwords.words('english')
def cleanAndNormalizeText(data):
    # tokenize
    tokenize = word_tokenize(data)
    # remove stop words
    filterText = [w for w in tokenize if w not in stop_words]
    filterText = [w for w in filterText if not len(w) <= 1]

    # stem
    ps = PorterStemmer()
    for i in range(len(filterText)-1):
        if len(filterText[i]) > 1:
            try:
                filterText[i] = ps.stem(filterText[i])
            except Exception as e:
                filterText[i] = filterText[i]

    return ' '.join(filterText)

In [13]:
# get data from csv
# check if need to pre-process description too

genderClassData = []
with open('C:\\Users\\abalata\\Downloads\\gender-classifier-DFE-791531.csv\\gender-classifier-DFE-791531.csv','r', encoding="utf8" ,errors="ignore") as csvfile:
    reader = csv.reader(csvfile)
    count = 0
    for row in reader:
        if count != 0:
            # description = cleanAndNormalizeText(row[10])
            text = cleanAndNormalizeText(row[19])
            genderClassData.append([row[0], row[5], text])
        count = count+1

Table = pd.DataFrame(genderClassData, columns=['ID', 'gender', 'text'])
# print(Table)


In [14]:
classDistribution = []
for c in ['male', 'female', 'brand']:
    classDistribution.append([c, Table["gender"].value_counts()[c]])
pd.DataFrame(classDistribution, columns=['Gender', '# of Samples'])

,Gender,# of Samples
0,male,6194
1,female,6700
2,brand,5942


In [15]:
n = 10
for c in ['male', 'female', 'brand']:
    print('Gender:', c)
    class_freq = pd.Series()
    class_df = Table.loc[Table["gender"] == c]
    print(class_df)
    # class_freq = class_freq.append(pd.Series(" ".join.class_df.split()).value_counts()[:n])
    # class_freq = class_freq.groupby(by=class_freq.index).sum().sort_values(ascending=False, axis=0)
    # print(class_freq.to_frame('freq'))

Gender: male
              ID gender                                               text
0      815719226   male  robbi respond To critic after win against eddi...
1      815719227   male  It felt like friend live stori them۝ http //t....
2      815719228   male    absolut ador loui start song hit hard feel good
3      815719229   male  Hi jordanspieth look url use ifttt Do n't typi...
7      815719233   male  gala bingo club bought 241m the UK 's largest ...
17     815719243   male  coolyazzy94 ditto 'm still learn favourit retw...
18     815719244   male   caribbro jstsaleem n't understand get server get
23     815719249   male  yall lmfaoo right when the choru came ON tear ...
24     815719250   male  jame bond premier night everymancinema oxt sid...
29     815719255   male  As oppos pump 's like HI hope you like do jump...
31     815719257   male  all magic hath No furi base real magick http /...
33     815719259   male  and got yard and point jet gave season http //...
35     81571

In [16]:
from sklearn.model_selection import train_test_split
# x = combined_news_djia_df[combined_news_djia_df.columns[2:4]]
x = Table.drop(['gender'], axis=1)
y = Table[["gender"]]
print(x)
print(y)

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.3, random_state=0)
xTrain = xTrain.apply(lambda x: " ".join(x), axis=1)
xTest = xTest.apply(lambda x: " ".join(x), axis=1)
xTrain = xTrain.tolist()
xTest = xTest.tolist()
yTrain = yTrain['gender'].tolist()
yTest = yTest['gender'].tolist()

              ID                                               text
0      815719226  robbi respond To critic after win against eddi...
1      815719227  It felt like friend live stori them۝ http //t....
2      815719228    absolut ador loui start song hit hard feel good
3      815719229  Hi jordanspieth look url use ifttt Do n't typi...
4      815719230  watch neighbour sky+ catch neighb xxx _____ُ_ԍ...
5      815719231  ive seen peopl train lamp chair tv etc http //...
6      815719232           bpackengin thank patienc take care issue
7      815719233  gala bingo club bought 241m the UK 's largest ...
8      815719234  _aphmau_ pic defin mcd fangirls/fanboy mcd shi...
9      815719235  evieladi love tree year never seen gorgeou aut...
10     815719236      take care http //t.co/orl9u1p8yi fitfluential
11     815719237  mtg deal 1x rank-up-mag the seventh one prio-e...
12     815719238                            just put ass line repay
13     815719239  http //t.co/z4sbwuugd8 what na

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
# The classifying process
def classify(parameters, feature, ml):
            pipeline = Pipeline(steps=[('vect', feature), ('clf', ml)])
            gs_clf = GridSearchCV(pipeline, parameters, n_jobs=1)
            gs_clf = gs_clf.fit(xTrain, yTrain)
            prediction = gs_clf.predict(xTest)
            accuracy = metrics.accuracy_score(yTest, prediction)
            return [accuracy, gs_clf.best_params_]

In [18]:
results = []
parameters = {'vect__max_df': (0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0), 'clf__alpha': (0.001, 0.01, 0.1, 1.0)}

# Using classify with Bag of Words and SVM algorithm
ans = classify(parameters, CountVectorizer(), SGDClassifier())
results.append(['Bag of Words', 'SVM', ans[1], ans[0]])

# Using classify with Bag of Words and NB algorithm
ans = classify(parameters, CountVectorizer(), MultinomialNB())
results.append(['Bag of Words', 'Naïve Bayes',  ans[1], ans[0]])

# Using classify with tf-idf and SVM algorithm
ans = classify(parameters, TfidfVectorizer(sublinear_tf=True, stop_words='english'), SGDClassifier())
results.append(['TF-IDF', 'SVM', ans[1], ans[0]])

# Using classify with tf-idf and NB algorithm
ans = classify(parameters, TfidfVectorizer(sublinear_tf=True, stop_words='english'), MultinomialNB())
results.append(['TF-IDF', 'Naïve Bayes', ans[1], ans[0]])

pd.DataFrame(results, columns=['feature extract', 'machine learning algorithm', 'parameters', 'accuracy'])

C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


...

C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


C:\Users\abalata\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


,feature extract,machine learning algorithm,parameters,accuracy
0,Bag of Words,SVM,"{'clf__alpha': 0.01, 'vect__max_df': 0.4}",0.517539
1,Bag of Words,Naïve Bayes,"{'clf__alpha': 1.0, 'vect__max_df': 0.4}",0.522527
2,TF-IDF,SVM,"{'clf__alpha': 0.001, 'vect__max_df': 0.6}",0.511721
3,TF-IDF,Naïve Bayes,"{'clf__alpha': 1.0, 'vect__max_df': 0.4}",0.502078
